# Context

Our bidding strategies run multiple optimization models during a single bidding time period, for instance for different expected prices. At the moment, this is done from scratch each time, but often only a few things from an optimization model change.

This notebook contains some explorations for persistent solvers
1. Example of using persistent solver when updating constraints on a model, and time it.
2. Example for model with updating time-indexed parameters in the objective function, as is done in the `MultiProfileBiddingStrategy`.


## 1. Simple model

In [5]:
import pyomo.environ as pyo

In [6]:
def create_model():
    """Create simple model."""
    m = pyo.ConcreteModel()
    m.x = pyo.Var()
    m.y = pyo.Var()
    m.obj = pyo.Objective(expr=m.x**2 + m.y**2)
    m.c = pyo.Constraint(expr=m.y >= -2 * m.x + 5)
    return m

In [7]:
def solve_model(model, solver="gurobi", show_result=True):
    """Solve simple model."""
    opt = pyo.SolverFactory(solver)
    if solver == "gurobi_persistent":
        save_results = False  # in this case, unclear what's better
        opt.set_instance(model)
        _ = opt.solve(save_results=save_results)
        model.c2 = pyo.Constraint(expr=m.y >= m.x)
        opt.add_constraint(m.c2)
        _ = opt.solve(save_results=save_results)
        opt.remove_constraint(m.c2)
        del m.c2
    else:
        _ = opt.solve(model)
        model.c2 = pyo.Constraint(expr=m.y >= m.x)
        _ = opt.solve(model)
        del m.c2

    if show_result:
        print(pyo.value(m.obj))

First, verify we get the same results

In [8]:
m = create_model()
res = solve_model(m, "gurobi_persistent")
res

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2684518
Academic license 2684518 - for non-commercial use only - registered to f.___@tudelft.nl
5.555555556979119


In [9]:
m = create_model()
res = solve_model(m, "gurobi")

5.555555556979119


Now, compare the timing

In [10]:
m = create_model()
%timeit res = solve_model(m, "gurobi_persistent", show_result=False)

11.8 ms ± 271 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
m = create_model()
%timeit res = solve_model(m, "gurobi", show_result=False)

18.7 ms ± 404 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
print("Before:")
m.pprint()
getattr(m, "obj")
delattr(m, "obj")
print("After deleting:")
m.pprint()

Before:
2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 1.666666666254618 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :  None : 1.6666666675057846 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x**2 + y**2

1 Constraint Declarations
    c : Size=1, Index=None, Active=True
        Key  : Lower : Body         : Upper : Active
        None :  -Inf : -2*x + 5 - y :   0.0 :   True

4 Declarations: x y obj c
After deleting:
2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 1.666666666254618 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : 

To generalize, we need to understand how we can modify different parts of a model


## 2. Docs
- [`GurobiPersistent`](https://pyomo.readthedocs.io/en/stable/reference/topical/solvers/gurobi_persistent.html#gurobipersistent)
- [`AbstractScalarVar`](https://pyomo.readthedocs.io/en/stable/api/pyomo.core.base.var.AbstractScalarVar.html#pyomo.core.base.var.AbstractScalarVar)
- Persistent Solver explanation: [link](https://pyomo.readthedocs.io/en/stable/explanation/solvers/persistent.html#persistent-solvers)

### From the Docs
- "Note that users are responsible for notifying the persistent solver interfaces when changes are made to the corresponding pyomo model."
- `GurobiPersistent` accepts only a ConcreteModel, not an abstract model (!)
- Key methods on the class
    - `add_block`, `remove_block`
    - `add_column`, -- **no method to remove columns**
    - `add_constraint`, `remove_constraint`
    - `add_var`, `remove_var`, `update_var`
    - `set_objective`
- It's necessary to first remove the component from the optimizer, then update the component in pyomo, and then add it back


For instance, constraints should be changed as follows

```python
m = pyo.ConcreteModel()
m.x = pyo.Var()
m.y = pyo.Var()
m.c = pyo.Constraint(expr=m.y >= -2*m.x + 5)
opt = pyo.SolverFactory('gurobi_persistent')
opt.set_instance(m)

opt.remove_constraint(m.c)
del m.c
m.c = pyo.Constraint(expr=m.y <= m.x)
opt.add_constraint(m.c)

```


An exception to this rule are variables, which we can update directly with the solver
```python
m = pyo.ConcreteModel()
m.x = pyo.Var()
m.y = pyo.Var()
m.c = pyo.Constraint(expr=m.y >= -2*m.x + 5)
opt = pyo.SolverFactory('gurobi_persistent')
opt.set_instance(m)

m.x.setlb(1.0) # first, update in pyomo - I understand this 
opt.update_var(m.x)
```

#### Indexed variables and constraints

To add and remove indexed variables and constraints, we need to iterate over their values:
```python
for v in indexed_var.values():
    opt.add_var(v)
    opt.remove_var(v)
for v in indexed_con.values():
    opt.add_constraint(v)
    opt.remove_constraint(v)
```




## 2. Time-indexed parameters to objective function, similar to MultiProfileBiddingStrategy

Explore how we can save time when creating and solving models: by avoiding creating instances of models that are very similar multiple times, and by re-using an existing solver instance. The example here is a minimal version of what we have in the `MultiProfileBiddingStrategy`.

In [13]:
parameters = {"first": {1: 0.5, 2: 3.4}, "second": {1: 5, 2: 3.4}}

In [14]:
def add_objective_to_model(model, name_of_obj_param="obj_params"):
    """Add an objective function to a model."""

    @model.Objective()
    def obj(model):
        obj_params = getattr(model, name_of_obj_param)
        return sum(obj_params[t] * model.x[t] ** 2 for t in model.time)

    return model


def create_model(params_obj_fct, param_method="decorator"):
    """Create a model with time index."""
    model = pyo.ConcreteModel()
    model.time = pyo.Set(initialize=[1, 2])
    model.x = pyo.Var(model.time)

    # Option 1: Define the parameter function
    if param_method == "decorator":

        @model.Param(model.time, domain=pyo.Any, mutable=False)
        def obj_params(model, t):
            return params_obj_fct[t]

    # Option 2: pas parameter attribute as string.
    # This allows (1) passing in a function to create the model,
    # and (2) letting the user define the name of the objective function
    # This is more general but currently we probably will not need it.
    elif param_method == "dynamic":

        def param_function(model, t):
            return params_obj_fct[t]

        param_with_rule = pyo.Param(model.time, domain=pyo.Any, mutable=False, rule=param_function)
        setattr(model, "obj_params", param_with_rule)

    model = add_objective_to_model(model)

    @model.Constraint()
    def c(model):
        return model.x[2] >= -2 * model.x[1] + 5

    return model


def update_model(model, params_obj_fct: dict):
    """Update parameters of obj fct in existing model."""
    delattr(model, "obj_params")
    delattr(model, "obj")

    @model.Param(model.time, domain=pyo.Any, mutable=True)
    def obj_params(model, t):
        return params_obj_fct[t]

    model = add_objective_to_model(model)

    return model

### (A) Baseline: solving twice

In [15]:
def solve_twice(params, show_results=True, param_method="decorator"):
    """Create and save a model twice."""
    model = create_model(params["first"], param_method=param_method)
    opt = pyo.SolverFactory("gurobi")
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))
    model = create_model(params["second"], param_method=param_method)
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))

In [16]:
solve_twice(parameters, param_method="decorator")

3.0141843971849
22.84946236650336


In [17]:
solve_twice(parameters, param_method="dynamic")

3.0141843971849
22.84946236650336


In [18]:
%timeit solve_twice(parameters, False, "decorator")

21.5 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit solve_twice(parameters, False, "dynamic")

20.2 ms ± 672 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### (B) Modifying the pyomo model

Update the pyomo model but send both to the solver separately

In [20]:
def solve_and_update(params, show_results=True, param_method="decorator"):
    """Create model, solve, change parameters send again to solver."""
    model = create_model(parameters["first"], param_method=param_method)
    opt = pyo.SolverFactory("gurobi")
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))

    model = update_model(model, parameters["second"])
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))

In [21]:
solve_and_update(parameters, param_method="decorator")

3.0141843971849
22.84946236650336


In [22]:
solve_and_update(parameters, param_method="dynamic")

3.0141843971849
22.84946236650336


In [23]:
%timeit solve_and_update(parameters, False, "decorator")

20.4 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit solve_and_update(parameters, False, "dynamic")

19.9 ms ± 616 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### (C) Using a persistent solver

In [25]:
def solve_persist_and_update(params, show_results=True, param_method="decorator"):
    """Create model, solve, change parameters only in solver, solve again."""
    model = create_model(parameters["first"], param_method=param_method)
    opt = pyo.SolverFactory("gurobi_persistent")
    opt.set_instance(model)
    _ = opt.solve(save_results=False)
    if show_results:
        print(pyo.value(model.obj))

    model = update_model(model, parameters["second"])
    opt.set_objective(model.obj)
    _ = opt.solve(save_results=False)
    if show_results:
        print(pyo.value(model.obj))

In [26]:
solve_persist_and_update(parameters, param_method="decorator")

3.0141843971849
22.84946236650336


In [27]:
solve_persist_and_update(parameters, param_method="dynamic")

3.0141843971849
22.84946236650336


In [28]:
%timeit solve_persist_and_update(parameters, False, "decorator")

13.5 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%timeit solve_persist_and_update(parameters, False, "dynamic")

13.6 ms ± 891 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Conclusion

- Overhead comes from sending model to the solver
- Creating (simple) abstract models "from scratch" vs modifying their parameters in place makes only a small difference
  - at least not with this simple example here. But when doing this repeatedly, it may add up?
- Relation to abstract vs concrete models: they don't interact with the solver. With this context, it makes sense that using a concrete vs abstract model makes little difference to the performance.

## Todo

Check "Repeated solves": https://pyomo.readthedocs.io/en/stable/howto/manipulating.html#repeated-solves

(1)https://pyomo.readthedocs.io/en/stable/howto/manipulating.html#activating-and-deactivating-objectives
One can have multiple objectives and activate them in sequences. This would allow to define the objectives once, and then iterate through them. Might be interesting to check the speed of this alternative.


(2) Transferring solutions out of the model object
```python
results = opt.solve(instance, load_solutions=False)
```
>This approach can be useful if there is a concern that the solver did not terminate with an optimal solution. 